<a href="https://colab.research.google.com/github/rogeliorv/encode-ai-bootcamp-g11/blob/main/Gethsun_WeekendI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [10]:
from openai import OpenAI
from google.colab import userdata
userdata.get('OPEN_API_KEY') # Retrieve your API key (replace 'OPEN_API_KEY' with the name you used to store it)

api_key = "OPEN_API_KEY"
client = OpenAI(api_key=api_key)

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a young and spirited Kenyan cook that loves to make Githeri. "
            "You help people by suggesting detailed recipes for dishes they want to cook, "
            "suggesting dishes based on ingredients provided, and criticizing recipes given by the user. "
            "You always try to be as clear as possible and provide the best possible suggestions for the user's needs. "
            "You know a lot about different cuisines and cooking techniques. You are also very patient and understanding with the user's needs and questions."
        ),
    }
]

user_input = input("Enter your request (ingredients, dish name, or recipe):\n")

# Determine the type of input
if ',' in user_input:
    input_type = "ingredients"
elif any(char.isdigit() for char in user_input):
    input_type = "recipe"
else:
    input_type = "dish"

# Respond based on input type
if input_type == "ingredients":
    messages.append(
        {
            "role": "user",
            "content": f"Suggest a dish name that can be made with these ingredients: {user_input}"
        }
    )
elif input_type == "dish":
    messages.append(
        {
            "role": "user",
            "content": f"Suggest me a detailed recipe and the preparation steps for making {user_input}"
        }
    )
elif input_type == "recipe":
    messages.append(
        {
            "role": "user",
            "content": f"Criticize this recipe and suggest changes: {user_input}"
        }
    )
else:
    print("Invalid input. Please enter a list of ingredients, a dish name, or a recipe.")
    exit()

model = "gpt-3.5-turbo"

stream = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

collected_messages = []
for chunk in stream:
    chunk_message = chunk.choices[0].delta.content or ""
    print(chunk_message, end="")
    collected_messages.append(chunk_message)

messages.append(
    {
        "role": "system",
        "content": "".join(collected_messages)
    }
)

while True:
    print("\n")
    user_input = input()
    # Determine the type of input
    if ',' in user_input:
        input_type = "ingredients"
    elif any(char.isdigit() for char in user_input):
        input_type = "recipe"
    else:
        input_type = "dish"

    # Respond based on input type
    if input_type == "ingredients":
        messages.append(
            {
                "role": "user",
                "content": f"Suggest a dish name that can be made with these ingredients: {user_input}"
            }
        )
    elif input_type == "dish":
        messages.append(
            {
                "role": "user",
                "content": f"Suggest me a detailed recipe and the preparation steps for making {user_input}"
            }
        )
    elif input_type == "recipe":
        messages.append(
            {
                "role": "user",
                "content": f"Criticize this recipe and suggest changes: {user_input}"
            }
        )
    else:
        print("Invalid input. Please enter a list of ingredients, a dish name, or a recipe.")
        continue

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    collected_messages = []
    for chunk in stream:
        chunk_message = chunk.choices[0].delta.content or ""
        print(chunk_message, end="")
        collected_messages.append(chunk_message)

    messages.append(
        {
            "role": "system",
            "content": "".join(collected_messages)
        }
    )